In [1]:
%load_ext autoreload
%autoreload 2

from tqdm import tqdm
import pandas as pd
import numpy as np
import gzip
import json
import math
from datetime import datetime
from IPython.display import display
%matplotlib inline

In [2]:
currency_pairs = ['USDT_BTC', 'BTC_ETH', 'BTC_XMR', 'BTC_XRP', 'BTC_FCT', 'BTC_NAV', 'BTC_DASH', 'BTC_MAID', 'BTC_ZEC']
currency_pair = currency_pairs[0]

In [3]:
folder = '../data/'
files = [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(folder)) for f in fn]
datafiles = sorted([f for f in files if f.endswith('.log.gz')])
print("Number of datafiles to extract from: {}".format(len(datafiles)))
print("First file: {}".format(datafiles[0]))
print("Last file : {}".format(datafiles[-1]))

Number of datafiles to extract from: 82405
First file: ../data/2016-11-08/poloniex_full_2016-11-08T10:00:09.955811.log.gz
Last file : ../data/2017-01-04/poloniex_full_2017-01-04T16:20:05.204740.log.gz


In [4]:
from manage_orderbooks import *
from orderbook_trade_simulator import *

In [5]:
filename = '../data/snapshots/orderbooks_USDT_BTC_range1.15_samplesNone_snapshot2016-12-15T15:09.dict'
orderbooks, timestamps = load_orderbook_snapshot(infile=filename, first_line=0, last_line=1000)
# display(timestamps)
print(len(orderbooks[0]))

# orderbooks = orderbooks[:1000]

100%|██████████| 1000/1000 [00:12<00:00, 78.45it/s]

Loaded Orderbooks: 1000
908


In [6]:
display(timestamps[0][:-10])

u'2016-11-08T10:00'

In [ ]:
ots = OrderbookTradingSimulator()

vol = 40
period_length = 15
lim = 706
order_periods = 10
must_trade = False

ob = orderbooks[0]

for i in range(order_periods):
    if i == order_periods-1:
        must_trade = True
    
    print("Period {}: volume={}, limit={}".format(i+1, vol, lim))
    new_ob = ots.trade_timespan(orderbooks[i*period_length:(i+1)*period_length],
                                timestamps[i*period_length:(i+1)*period_length],
                                timespan=period_length, must_trade=must_trade,
                                volume=vol, limit=lim, verbose=True)
    vol = ots.history.volume_left.values[-1]
    print("vol", vol)
    print("volume_of_last_trade_period {}".format(ots.volume_of_last_trade_period))
    
    if abs(vol) == 0:
        print("Done after {} order_periods".format(i+1))
        break
    elif vol > 0:
        pass
        #lim += 1
    elif vol < 0:
        pass
        #lim -= 1
ots.summarize(new_ob)
    

# new_ob = ots.trade_timespan(orderbooks[timespan:timespan+timespan], volume=vol, limit=706, verbose=True, timespan=timespan)
plot_orderbook(new_ob, title = "Orderbook {}".format(ots.t), normalized=True, range_factor=1.05)

 13%|█▎        | 2/15 [00:00<00:00, 19.44it/s]

Period 1: volume=40, limit=706


100%|██████████| 15/15 [00:01<00:00,  8.91it/s]


,ASK,BID,CENTER,LIMIT,SPREAD,TIMESPAN,VOLUME,avg,cashflow,high,low,slippage,volume_left,volume_traded
2016-11-08T10:00,705.45,705.0,705.224976,706,0.45,15,40,705.449999,-4838.059048,705.45,705.0,0.000005,33.141882,6.858118


 #####   TRADE Number 15   #####
Purchase history


...

Last trade


{}

Last trade  : 0.0000/32.2154 shares
Total trades: 7.7846/40.0000 shares
Spread: 0.96 (bid: 705.12, ask: 706.08)
Adjusted Orderbook:


,Amount,Price,Type,Volume,VolumeAcc,norm_Price
997,1.598709,704.580000,bid,1126.418521,3450.118887,0.998555
998,1.600500,704.790000,bid,1128.016395,2323.700366,0.998852
999,1.695717,705.120000,bid,1195.683971,1195.683971,0.999320
1000,0.000000,705.599891,center,0.000000,0.000000,1.000000
1001,2.541509,706.080000,ask,1794.508992,1794.508992,1.000680
1002,0.006882,706.180000,ask,4.859910,1799.368902,1.000822
1003,0.508136,706.190000,ask,358.840512,2158.209414,1.000836


  0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
ots = OrderbookTradingSimulator()

vol = 2

ob = orderbooks[0]
new_ob = ots.trade(orderbooks[0], volume=vol, limit=None, verbose=True)
plot_orderbook(new_ob, title = "Orderbook {}".format(ots.t), normalized=True, range_factor=1.05)
trade_counter = 1

In [ ]:
center = orderbooks[0][orderbooks[0].Type=='center'].Price.values[0]
bid = orderbooks[0][orderbooks[0].Type=='bid'].Price.values[-1]
ask = orderbooks[0][orderbooks[0].Type=='ask'].Price.values[0]
print("Bid:    {}".format(bid))
print("Center: {}".format(center))
print("Ask:    {}\n".format(ask))

result = 0
trade_vol = -10
limit=704
trade_info = place_order(orderbooks[0], trade_vol, limit=limit)
cash_flow = trade_info['cashflow']
trade_vol -= trade_info['amount_fulfilled']

display(trade_info)
display(orderbook_preview(ob, 3))

#cash_flow = 0
minute = 1
for i, ob in enumerate(orderbooks[1:]):
    if abs(trade_vol) < 1.0e-10:
        break
        
    
    print("timesteps: {} ==> {}, {}".format(minute, cash_flow, trade_vol))
    if abs(trade_vol) > 1.0e-10 and i == len(orderbooks[1:])-1:
        print("TRADE!!!")
        # trade always in last Timestep
        trade_info = place_order(ob,  trade_vol, limit=None, trade_history=trade_info['trade_history'])
        cash_flow += trade_info['cashflow']
        trade_vol -= trade_info['amount_fulfilled']
    
    elif abs(trade_vol) > 1.0e-10:
        print("trade_vol", trade_vol)
        trade_info = place_order(ob,  trade_vol, limit=limit, trade_history=trade_info['trade_history'])
        cash_flow += trade_info['cashflow']
        trade_vol -= trade_info['amount_fulfilled']

    display(trade_info)
    
    minute += 1
    
    print("")
    
    print("trade_vol: {:1.4f}, center: {:1.4f}, limit: {}".format(trade_vol, ob[ob.Type=='center'].Price.values[0], limit))
print("timesteps: {} ==> {}, {}".format(minute, cash_flow, trade_vol))

display(orderbook_preview(ob, 3))

In [ ]:
range_factor = 1.01
# plot_orderbook(data[0], normalized=True, range_factor=range_factor)
print(len(orderbooks))
for idx in [1, 5000, 25000, -1]:
    print(idx, timestamps[idx])
    display(orderbook_preview(orderbooks[idx], 3))
    plot_orderbook(orderbooks[idx], title=timestamps[idx], normalized=True, range_factor=range_factor)